In [1]:
import sys
sys.path.append('../')
from kan import KAN

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
import torch
import sympy as sp
from mpmath import mp, mpf

In [3]:
data = pd.read_excel("./Dry_Bean_Dataset/Dry_Bean_Dataset.xlsx")
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [4]:
data["Class"].value_counts()

Class
DERMASON    3546
SIRA        2636
SEKER       2027
HOROZ       1928
CALI        1630
BARBUNYA    1322
BOMBAY       522
Name: count, dtype: int64

In [5]:
data.shape

(13611, 17)

In [6]:
data.isna().sum()

Area               0
Perimeter          0
MajorAxisLength    0
MinorAxisLength    0
AspectRation       0
Eccentricity       0
ConvexArea         0
EquivDiameter      0
Extent             0
Solidity           0
roundness          0
Compactness        0
ShapeFactor1       0
ShapeFactor2       0
ShapeFactor3       0
ShapeFactor4       0
Class              0
dtype: int64

In [7]:
features = data.drop("Class", axis=1)
target = data["Class"]

In [8]:
target

0           SEKER
1           SEKER
2           SEKER
3           SEKER
4           SEKER
           ...   
13606    DERMASON
13607    DERMASON
13608    DERMASON
13609    DERMASON
13610    DERMASON
Name: Class, Length: 13611, dtype: object

In [9]:
features = data.drop("Class", axis=1)
target = data["Class"]

le = LabelEncoder()
target_encoded = le.fit_transform(target)

scaler = StandardScaler() # .set_output(transform="pandas")
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled,
                                                    target_encoded,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=target_encoded)

In [10]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
device = 'cuda'

print(device)

cuda


In [11]:
def check_accuracy(model: KAN, dataset: dict):
    return torch.mean((torch.round(model(dataset['train_input'])[:,0]) == dataset['train_label'][:,0]).float()), torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float())

In [12]:
dataset = {}
dataset['train_input'] = torch.from_numpy(X_train).to(device)
dataset['test_input'] = torch.from_numpy(X_test).to(device)
dataset['train_label'] = torch.from_numpy(y_train).long().to(device)
dataset['test_label'] = torch.from_numpy(y_test).long().to(device)

In [13]:
print(dataset['train_input'].shape)
print(dataset['train_label'].shape)
print(dataset['test_input'].shape)
print(dataset['test_label'].shape)

torch.Size([10888, 16])
torch.Size([10888])
torch.Size([2723, 16])
torch.Size([2723])


In [ ]:
model = KAN(width=[16,2,7], grid=6, k=3, device=device)

def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).float())

results = model.train(dataset, opt="LBFGS", steps=200, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss(), lr=0.0002, device=device)
print("Train accuracy:", results['train_acc'][-1], " - Test accuracy:", results['test_acc'][-1])

In [15]:
model.suggest_symbolic(1,0,0)
model.suggest_symbolic(1,0,2)
model.suggest_symbolic(0,8,1)
model.suggest_symbolic(0,14,1)
model.suggest_symbolic(0,8,0)
model.suggest_symbolic(1,1,5)

function , r2
custom_26 , 0.9994375526942431
custom_30 , 0.993428505085262
custom_10 , 0.9921494273071799
gaussian , 0.9917480311428325
custom_15 , 0.9905268123035471
function , r2
custom_27 , 0.9996031019090259
custom_28 , 0.9964652523892031
custom_30 , 0.9959753069792274
custom_25 , 0.9955090156022409
custom_26 , 0.995241698805847
function , r2
custom_2 , 0.9940314081188463
custom_28 , 0.9913784901958307
custom_23 , 0.934583998585195
custom_10 , 0.8911658415530632
custom_30 , 0.8856934983471046
function , r2
custom_29 , 0.9942193529454237
custom_5 , 0.9929932214215718
custom_6 , 0.9694759575911313
custom_13 , 0.9655861938752512
custom_1 , 0.9579162155043786
function , r2
custom_30 , 0.9993073912630392
custom_7 , 0.9915616371483578
custom_8 , 0.9546515829966041
custom_4 , 0.936722503307291
custom_25 , 0.9301118310176558
function , r2
custom_33 , 0.9999948500903327
custom_23 , 0.9998912870787362
custom_9 , 0.9997792096968321
custom_17 , 0.9996546345720452
custom_7 , 0.9996110503668516


('custom_33',
 (<function kan.utils.<lambda>(x)>, <function kan.utils.<lambda>(x)>),
 0.9999948500903327)

In [16]:
model.suggest_symbolic(0,9,0)
model.suggest_symbolic(0,12,0)
model.suggest_symbolic(0,15,1)

function , r2
custom_10 , 0.9908389964293289
custom_28 , 0.9888288637939551
custom_8 , 0.9671456837538552
custom_2 , 0.9628077698035069
custom_11 , 0.9598863111144825
function , r2
custom_29 , 0.9961651193839038
custom_2 , 0.9952222371523524
custom_9 , 0.9948717584101717
custom_14 , 0.9944936082625115
custom_26 , 0.9942060090963013
function , r2
custom_15 , 0.9959218935426571
custom_16 , 0.9956181025442119
arctanh , 0.9955252161645253
arcsin , 0.9954627195611794
sqrt , 0.995365169729193


('custom_15',
 (<function kan.utils.<lambda>(x)>, <function kan.utils.<lambda>(x)>),
 0.9959218935426571)

In [17]:
model.fix_symbolic(0,1,1,'custom_0')
model.fix_symbolic(0,4,1,'custom_1')
model.fix_symbolic(0,8,1,'custom_2')
model.fix_symbolic(0,10,1,'custom_3')
model.fix_symbolic(0,11,1,'custom_4')
model.fix_symbolic(0,14,1,'custom_5')
model.fix_symbolic(0,5,0,'custom_6')
# model.fix_symbolic(0,8,0,'custom_7')
model.fix_symbolic(0,15,0,'custom_8')
model.fix_symbolic(1,0,1,'custom_9')
model.fix_symbolic(0,9,0,'custom_10')
model.fix_symbolic(1,0,6,'custom_11')
model.fix_symbolic(1,1,6,'custom_12')
model.fix_symbolic(0,2,1,'custom_13')
model.fix_symbolic(0,3,0,'custom_14')
model.fix_symbolic(0,3,1,'custom_15')
model.fix_symbolic(0,7,0,'custom_16')
model.fix_symbolic(0,7,1,'custom_17')
model.fix_symbolic(0,12,0,'custom_2')
model.fix_symbolic(0,12,1,'custom_19')

model.fix_symbolic(0,13,0,'custom_20')
model.fix_symbolic(0,14,0,'custom_19')
model.fix_symbolic(1,0,3,'custom_19')
model.fix_symbolic(1,0,5,'custom_23')
model.fix_symbolic(1,1,1,'custom_2')
model.fix_symbolic(1,1,2,'custom_25')

model.fix_symbolic(0,0,1,'custom_0')
model.fix_symbolic(0,1,0,'custom_20')
model.fix_symbolic(0,6,1,'custom_10')
model.fix_symbolic(0,10,0,'custom_4')

model.fix_symbolic(1,0,0,'custom_26')
model.fix_symbolic(1,0,2,'custom_27')
# model.fix_symbolic(0,8,1,'custom_28')
# model.fix_symbolic(0,14,1,'custom_29')
model.fix_symbolic(0,8,0,'custom_30')
# # model.fix_symbolic(0,9,0,'custom_31')
# # model.fix_symbolic(0,12,0,'custom_25')
# # model.fix_symbolic(0,15,1,'custom_25')
model.fix_symbolic(1,1,5,'custom_33')

r2 is 0.9993736765199082
r2 is 0.9976477685163895
r2 is 0.9940314081188463
r2 is 0.999414909783709
r2 is 0.9970250804390575
r2 is 0.9929932214215718
r2 is 0.9984417285188715
r2 is 0.9990650474790466
r2 is 0.9994690829671827
r2 is 0.9908389964293289
r2 is 0.9961581969619719
r2 is 0.9998205955592893
r2 is 0.9969907206687609
r2 is 0.9995506480505414
r2 is 0.9992690062367277
r2 is 0.9977459724989439
r2 is 0.9996770031123737
r2 is 0.9952222371523524
r2 is 0.99952642391605
r2 is 0.9996529382250956
r2 is 0.9973711609710695
r2 is 0.9996201049260488
r2 is 0.9997514195211953
r2 is 0.9995351037375619
r2 is 0.9998370572123041
r2 is 0.9986143147230669
r2 is 0.9980154608245461
r2 is 0.997481030659041
r2 is 0.9990777842095301
r2 is 0.9994375526942431
r2 is 0.9996031019090259
r2 is 0.9993073912630392
r2 is 0.9999948500903327


tensor(1.0000, device='cuda:0', dtype=torch.float64, grad_fn=<SelectBackward0>)

In [18]:
lib = ['x','x^2','x^3','1/x','sqrt','1/sqrt(x)','exp','log','abs','sin','tan','tanh','sigmoid','relu','sgn','arcsin','arctan','arctanh','0','gaussian','cosh']
model.auto_symbolic(lib=lib)
# model.auto_symbolic()

fixing (0,0,0) with gaussian, r2=0.9981476565455721
skipping (0,0,1) since already symbolic
skipping (0,1,0) since already symbolic
skipping (0,1,1) since already symbolic
fixing (0,2,0) with gaussian, r2=0.9997525012383789
skipping (0,2,1) since already symbolic
skipping (0,3,0) since already symbolic
skipping (0,3,1) since already symbolic
fixing (0,4,0) with arctan, r2=0.9982932162605482
skipping (0,4,1) since already symbolic
skipping (0,5,0) since already symbolic
fixing (0,5,1) with tanh, r2=0.9994966763516928
fixing (0,6,0) with relu, r2=0.9977310296337548
skipping (0,6,1) since already symbolic
skipping (0,7,0) since already symbolic
skipping (0,7,1) since already symbolic
skipping (0,8,0) since already symbolic
skipping (0,8,1) since already symbolic
skipping (0,9,0) since already symbolic
fixing (0,9,1) with gaussian, r2=0.9965646999348321
skipping (0,10,0) since already symbolic
skipping (0,10,1) since already symbolic
fixing (0,11,0) with sin, r2=0.9987168224211662
skipping

In [19]:
print("Train accuracy:", train_acc(), " - Test accuracy:", test_acc())

Train accuracy: tensor(0.9265, device='cuda:0')  - Test accuracy: tensor(0.9243, device='cuda:0')


In [20]:
model.save_ckpt('test_7_dry_bean_formula_symbolic.ckpt')

save this model to ./model_ckpt/test_7_dry_bean_formula_symbolic.ckpt


In [21]:
formula1, formula2, formula3, formula4, formula5, formula6, formula7 = model.symbolic_formula()[0]

In [22]:
print('Formula1:', formula1)
print('Formula2:', formula2)
print('Formula3:', formula3)
print('Formula4:', formula4)
print('Formula5:', formula5)
print('Formula6:', formula6)
print('Formula7:', formula7)

Formula1: 1.0*(0.97 - sin(sin(0.73*(0.55*sin(0.5*x_6) + 0.24)*log(sin(1.01*x_6 + 0.01) + 1.42) - 2.72*log(log(0.24*sin(0.91*exp(asinh(asinh(0.54*x_13 + 0.83)/cos(cos(0.13*exp(-0.54*x_13)))))) + 1) + 1) - 1.31*sin(0.33*x_12 + 5.14) + 0.74*sin(Abs(log(asinh(cos(log(Abs(1.01*x_10 - 0.41) + 1))) + 1)**3 + 0.14) - 0.11) - 0.78*sin(sin(0.98*x_8 - 1.27))**20 + 0.73*tan(1.76*atan(sin((-0.09*x_16 - sin(0.24*x_16 + 0.01)**3 - 0.59)**3)) + 3.02) - 0.73*tan(0.09*x_9 + 0.23*sin(-1.33*x_9 + 1.33*sin(1.0*x_9 - 0.34) + 0.93) - 0.03) - 0.78*tan(sinh(tanh(asinh(0.98*x_8 - 0.03) + 0.09)**7)) + 0.75*tan(asinh(atan(0.43*x_4 + 0.22*sin(0.91*x_4)**4 + 0.12)**2 - 0.06)) - 0.05*sinh(1.01*x_14 + 0.02) + 0.04*sinh(1.33*x_2 - 0.35) + 0.05*cosh(1.01*x_14 + 0.02) - 0.04*cosh(1.33*x_2 - 0.35) + 1.47*tanh((0.35*(0.52*x_11 + 1)**2/sqrt(0.27*(0.52*x_11 + 1)**4 + 1) - 0.4)*tanh(sinh(0.37*x_11 + 0.72) + 0.68) + 0.12) + 0.73*tanh(sinh(sinh(1.32*x_14 + 0.03))) - 0.59*tanh(sinh(sinh(1.72*x_2 - 0.46))) + 0.05*asinh(1.03*(x_1

In [23]:
def import_formula_from_txt(path):
    with open(path, 'r') as f:
        import_expr = f.read()
    loaded_expr = sp.sympify(import_expr)
    return loaded_expr

def export_formula_to_txt(formula, path):
    export_expr = sp.srepr(formula)
    with open(path, 'w') as f:
        f.write(export_expr)

In [24]:
export_formula_to_txt(formula1, './formula1_7class.txt')
export_formula_to_txt(formula2, './formula2_7class.txt')
export_formula_to_txt(formula3, './formula3_7class.txt')
export_formula_to_txt(formula4, './formula4_7class.txt')
export_formula_to_txt(formula5, './formula5_7class.txt')
export_formula_to_txt(formula6, './formula6_7class.txt')
export_formula_to_txt(formula7, './formula7_7class.txt')

loaded_formula1 = import_formula_from_txt('./formula1_7class.txt')
loaded_formula2 = import_formula_from_txt('./formula2_7class.txt')
loaded_formula3 = import_formula_from_txt('./formula3_7class.txt')
loaded_formula4 = import_formula_from_txt('./formula4_7class.txt')
loaded_formula5 = import_formula_from_txt('./formula5_7class.txt')
loaded_formula6 = import_formula_from_txt('./formula6_7class.txt')
loaded_formula7 = import_formula_from_txt('./formula7_7class.txt')

print('Formula1:', loaded_formula1)
print('Formula2:', loaded_formula2)
print('Formula3:', loaded_formula3)
print('Formula4:', loaded_formula4)
print('Formula5:', loaded_formula5)
print('Formula6:', loaded_formula6)
print('Formula7:', loaded_formula7)

Formula1: 1.0*(0.97 - sin(sin(0.73*(0.55*sin(0.5*x_6) + 0.24)*log(sin(1.01*x_6 + 0.01) + 1.42) - 2.72*log(log(0.24*sin(0.91*exp(asinh(asinh(0.54*x_13 + 0.83)/cos(cos(0.13*exp(-0.54*x_13)))))) + 1) + 1) - 1.31*sin(0.33*x_12 + 5.14) + 0.74*sin(Abs(log(asinh(cos(log(Abs(1.01*x_10 - 0.41) + 1))) + 1)**3 + 0.14) - 0.11) - 0.78*sin(sin(0.98*x_8 - 1.27))**20 + 0.73*tan(1.76*atan(sin((-0.09*x_16 - sin(0.24*x_16 + 0.01)**3 - 0.59)**3)) + 3.02) - 0.73*tan(0.09*x_9 + 0.23*sin(-1.33*x_9 + 1.33*sin(1.0*x_9 - 0.34) + 0.93) - 0.03) - 0.78*tan(sinh(tanh(asinh(0.98*x_8 - 0.03) + 0.09)**7)) + 0.75*tan(asinh(atan(0.43*x_4 + 0.22*sin(0.91*x_4)**4 + 0.12)**2 - 0.06)) - 0.05*sinh(1.01*x_14 + 0.02) + 0.04*sinh(1.33*x_2 - 0.35) + 0.05*cosh(1.01*x_14 + 0.02) - 0.04*cosh(1.33*x_2 - 0.35) + 1.47*tanh((0.35*(0.52*x_11 + 1)**2/sqrt(0.27*(0.52*x_11 + 1)**4 + 1) - 0.4)*tanh(sinh(0.37*x_11 + 0.72) + 0.68) + 0.12) + 0.73*tanh(sinh(sinh(1.32*x_14 + 0.03))) - 0.59*tanh(sinh(sinh(1.72*x_2 - 0.46))) + 0.05*asinh(1.03*(x_1

In [25]:
variables = loaded_formula1.free_symbols
print("Variables in the expression:", variables)

Variables in the expression: {x_2, x_9, x_13, x_10, x_14, x_1, x_5, x_16, x_3, x_7, x_15, x_11, x_6, x_4, x_12, x_8}


In [26]:
def acc_new(f_sympy_1, f_sympy_2, f_sympy_3, f_sympy_4, f_sympy_5, f_sympy_6, f_sympy_7, X, y):
    batch = X.shape[0]
    correct = 0
    res = []

    x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16 = sp.symbols('x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16')

    f_math_1 = sp.lambdify((x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16), f_sympy_1, modules=['mpmath'])
    f_math_2 = sp.lambdify((x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16), f_sympy_2, modules=['mpmath'])
    f_math_3 = sp.lambdify((x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16), f_sympy_3, modules=['mpmath'])
    f_math_4 = sp.lambdify((x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16), f_sympy_4, modules=['mpmath'])
    f_math_5 = sp.lambdify((x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16), f_sympy_5, modules=['mpmath'])
    f_math_6 = sp.lambdify((x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16), f_sympy_6, modules=['mpmath'])
    f_math_7 = sp.lambdify((x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16), f_sympy_7, modules=['mpmath'])
    
    mp.dps = 30
    
    for i in range(batch):
        # if(i%100 == 0):
        #     print("==========", i, " ==========")

        logit1 = f_math_1(mpf(str(X[i,0].item())), mpf(str(X[i,1].item())), mpf(str(X[i,2].item())), mpf(str(X[i,3].item())), mpf(str(X[i,4].item())), mpf(str(X[i,5].item())), mpf(str(X[i,6].item())), mpf(str(X[i,7].item())), mpf(str(X[i,8].item())), mpf(str(X[i,9].item())), mpf(str(X[i,10].item())), mpf(str(X[i,11].item())), mpf(str(X[i,12].item())), mpf(str(X[i,13].item())), mpf(str(X[i,14].item())), mpf(str(X[i,15].item())))
        logit2 = f_math_2(mpf(str(X[i,0].item())), mpf(str(X[i,1].item())), mpf(str(X[i,2].item())), mpf(str(X[i,3].item())), mpf(str(X[i,4].item())), mpf(str(X[i,5].item())), mpf(str(X[i,6].item())), mpf(str(X[i,7].item())), mpf(str(X[i,8].item())), mpf(str(X[i,9].item())), mpf(str(X[i,10].item())), mpf(str(X[i,11].item())), mpf(str(X[i,12].item())), mpf(str(X[i,13].item())), mpf(str(X[i,14].item())), mpf(str(X[i,15].item())))
        logit3 = f_math_3(mpf(str(X[i,0].item())), mpf(str(X[i,1].item())), mpf(str(X[i,2].item())), mpf(str(X[i,3].item())), mpf(str(X[i,4].item())), mpf(str(X[i,5].item())), mpf(str(X[i,6].item())), mpf(str(X[i,7].item())), mpf(str(X[i,8].item())), mpf(str(X[i,9].item())), mpf(str(X[i,10].item())), mpf(str(X[i,11].item())), mpf(str(X[i,12].item())), mpf(str(X[i,13].item())), mpf(str(X[i,14].item())), mpf(str(X[i,15].item())))
        logit4 = f_math_4(mpf(str(X[i,0].item())), mpf(str(X[i,1].item())), mpf(str(X[i,2].item())), mpf(str(X[i,3].item())), mpf(str(X[i,4].item())), mpf(str(X[i,5].item())), mpf(str(X[i,6].item())), mpf(str(X[i,7].item())), mpf(str(X[i,8].item())), mpf(str(X[i,9].item())), mpf(str(X[i,10].item())), mpf(str(X[i,11].item())), mpf(str(X[i,12].item())), mpf(str(X[i,13].item())), mpf(str(X[i,14].item())), mpf(str(X[i,15].item())))
        logit5 = f_math_5(mpf(str(X[i,0].item())), mpf(str(X[i,1].item())), mpf(str(X[i,2].item())), mpf(str(X[i,3].item())), mpf(str(X[i,4].item())), mpf(str(X[i,5].item())), mpf(str(X[i,6].item())), mpf(str(X[i,7].item())), mpf(str(X[i,8].item())), mpf(str(X[i,9].item())), mpf(str(X[i,10].item())), mpf(str(X[i,11].item())), mpf(str(X[i,12].item())), mpf(str(X[i,13].item())), mpf(str(X[i,14].item())), mpf(str(X[i,15].item())))
        logit6 = f_math_6(mpf(str(X[i,0].item())), mpf(str(X[i,1].item())), mpf(str(X[i,2].item())), mpf(str(X[i,3].item())), mpf(str(X[i,4].item())), mpf(str(X[i,5].item())), mpf(str(X[i,6].item())), mpf(str(X[i,7].item())), mpf(str(X[i,8].item())), mpf(str(X[i,9].item())), mpf(str(X[i,10].item())), mpf(str(X[i,11].item())), mpf(str(X[i,12].item())), mpf(str(X[i,13].item())), mpf(str(X[i,14].item())), mpf(str(X[i,15].item())))
        logit7 = f_math_7(mpf(str(X[i,0].item())), mpf(str(X[i,1].item())), mpf(str(X[i,2].item())), mpf(str(X[i,3].item())), mpf(str(X[i,4].item())), mpf(str(X[i,5].item())), mpf(str(X[i,6].item())), mpf(str(X[i,7].item())), mpf(str(X[i,8].item())), mpf(str(X[i,9].item())), mpf(str(X[i,10].item())), mpf(str(X[i,11].item())), mpf(str(X[i,12].item())), mpf(str(X[i,13].item())), mpf(str(X[i,14].item())), mpf(str(X[i,15].item())))

        res.append(np.array([logit1, logit2, logit3, logit4, logit5, logit6, logit7]))
        correct += np.argmax([logit1, logit2, logit3, logit4, logit5, logit6, logit7]) == y[i]

    return correct/batch, res

In [ ]:
acc, res = acc_new(loaded_formula1, loaded_formula2, loaded_formula3, loaded_formula4, loaded_formula5, loaded_formula6, loaded_formula7, dataset['test_input'], dataset['test_label'])

print('Test acc of the formula:', acc)